In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

tensor([[0.0984, 0.1052, 0.0904, 0.0982, 0.0976, 0.1054, 0.0996, 0.0948, 0.1078,
         0.1024]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([8], device='cuda:0')


In [10]:
input_image = torch.rand(3, 28, 28)
print(input_image)

tensor([[[0.0436, 0.5993, 0.9709,  ..., 0.3266, 0.9071, 0.7269],
         [0.3657, 0.8568, 0.3252,  ..., 0.2520, 0.8001, 0.6722],
         [0.8622, 0.0724, 0.9530,  ..., 0.7771, 0.8177, 0.6770],
         ...,
         [0.5521, 0.9809, 0.7461,  ..., 0.7385, 0.6298, 0.0640],
         [0.8184, 0.5216, 0.6353,  ..., 0.8064, 0.9803, 0.6261],
         [0.7694, 0.9395, 0.8859,  ..., 0.8938, 0.1231, 0.6096]],

        [[0.1835, 0.6387, 0.3913,  ..., 0.0134, 0.1164, 0.8134],
         [0.8036, 0.4733, 0.1200,  ..., 0.0372, 0.3225, 0.0781],
         [0.8932, 0.1416, 0.5404,  ..., 0.9910, 0.1004, 0.1376],
         ...,
         [0.6012, 0.8455, 0.3162,  ..., 0.9162, 0.9572, 0.8898],
         [0.1048, 0.1021, 0.9770,  ..., 0.0452, 0.6355, 0.8164],
         [0.7217, 0.7310, 0.8383,  ..., 0.8126, 0.6501, 0.2249]],

        [[0.3677, 0.8471, 0.7433,  ..., 0.5982, 0.4485, 0.2796],
         [0.1205, 0.1173, 0.9664,  ..., 0.1094, 0.9274, 0.1865],
         [0.4675, 0.3483, 0.8027,  ..., 0.5601, 0.9174, 0.

In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [12]:
layer1 = nn.Linear(in_features=28 * 28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [13]:
print(f'Before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

Before ReLU: tensor([[ 0.0199, -0.4441, -0.0498, -0.0816,  0.2960,  0.5963,  0.2349, -0.0016,
         -0.0313, -0.1073, -0.0041, -0.6117,  0.1677,  0.1700,  0.1472,  0.1582,
          0.0380, -0.2044, -0.7704,  0.0208],
        [-0.0198, -0.6509, -0.1629, -0.2935,  0.2793,  0.2847,  0.1448, -0.1801,
          0.1436, -0.1255, -0.2712, -0.5783,  0.1884,  0.2422,  0.2382,  0.2403,
         -0.4527, -0.3748, -0.1755, -0.0629],
        [ 0.3111, -0.3335,  0.0283, -0.2328,  0.2180,  0.2552,  0.1493, -0.5543,
         -0.0991, -0.1948, -0.1090, -0.5378,  0.0039,  0.1547,  0.1791,  0.1084,
         -0.1567, -0.3375, -0.3306,  0.0365]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0199, 0.0000, 0.0000, 0.0000, 0.2960, 0.5963, 0.2349, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.1677, 0.1700, 0.1472, 0.1582, 0.0380, 0.0000,
         0.0000, 0.0208],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.2793, 0.2847, 0.1448, 0.0000, 0.1436,
         0.0000, 0.0000, 0.0000, 0.1884, 0.2422, 0.23

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [15]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [22]:
print(f'Model structure: {model}\n\n')

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n')

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0189, -0.0336,  0.0173,  ...,  0.0257, -0.0056,  0.0046],
        [-0.0063, -0.0057, -0.0176,  ...,  0.0104, -0.0213,  0.0341]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0267, -0.0045], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0032,  0.0313,  0.0362,  ..., -0.0159,  0.0052,  0.0373],
        [-0.0236, -0.0116,  0.0253,  ..., -0.0116, -0.0414, -0.0333]],
       device='cuda:0', grad_fn=<Slice